# Setup of true test

Note: This version is against the true solution, so adaptive timestepping is used. This means that the option which is set to get the correct timepoints is saveat, not tstops!

In [1]:
using ParameterizedFunctions, OrdinaryDiffEq, DiffEqParamEstim
using BlackBoxOptim, NLopt, Plots

INFO: Recompiling stale cache file C:\Users\Denis\.julia\lib\v0.6\Roots.ji for module Roots.
INFO: Recompiling stale cache file C:\Users\Denis\.julia\lib\v0.6\OrdinaryDiffEq.ji for module OrdinaryDiffEq.


In [2]:
Xiang2015Bounds = Tuple{Float64, Float64}[(9, 11), (20, 30), (2, 3)] # for local optimizations
xlow_bounds = [9.0,20.0,2.0]
xhigh_bounds = [11.0,30.0,3.0]
LooserBounds = Tuple{Float64, Float64}[(0, 22), (0, 60), (0, 6)] # for global optimization
GloIniPar = [0.0, 0.5, 0.1] # for global optimizations
LocIniPar = [9.0, 20.0, 2.0] # for local optimization

3-element Array{Float64,1}:
  9.0
 20.0
  2.0

In [3]:
g1 = @ode_def_bare LorenzExample begin
  dx = σ*(y-x)
  dy = x*(ρ-z) - y
  dz = x*y - β*z
    end σ=>10.0 ρ=>28.0 β=>2.66    # Parameters used to construct the dataset

r0 = [1.0; 0.0; 0.0]                #[-11.8,-5.1,37.5] PODES Initial values of the system in space # [0.1, 0.0, 0.0]
tspan = (0.0, 50.0)                 # PODES sample of 3000 observations over the (0,30) timespan
prob = ODEProblem(g1, r0, tspan)
tspan2 = (0.0, 3.0)                 # Xiang test sample of 300 observations with a timestep of 0.01
prob_short = ODEProblem(g1, r0, tspan2)

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 3.0)
u0: [1.0, 0.0, 0.0]

In [4]:
dt = 30.0/3000                    
tf = 30.0
tinterval = 0:dt:tf
t  = collect(tinterval)

3001-element Array{Float64,1}:
  0.0 
  0.01
  0.02
  0.03
  0.04
  0.05
  0.06
  0.07
  0.08
  0.09
  0.1 
  0.11
  0.12
  ⋮   
 29.89
 29.9 
 29.91
 29.92
 29.93
 29.94
 29.95
 29.96
 29.97
 29.98
 29.99
 30.0 

In [5]:
h = 0.01
M = 300
tstart = 0.0
tstop = tstart + M * h
tinterval_short = 0:h:tstop
t_short = collect(tinterval_short)

301-element Array{Float64,1}:
 0.0 
 0.01
 0.02
 0.03
 0.04
 0.05
 0.06
 0.07
 0.08
 0.09
 0.1 
 0.11
 0.12
 ⋮   
 2.89
 2.9 
 2.91
 2.92
 2.93
 2.94
 2.95
 2.96
 2.97
 2.98
 2.99
 3.0 

In [6]:
# Generate Data
data_sol_short = solve(prob_short,Vern9(),saveat=t_short,reltol=1e-9,abstol=1e-9)
data_short = convert(Array, data_sol_short) # This operation produces column major dataset obs as columns, equations as rows
data_sol = solve(prob,Vern9(),saveat=t,reltol=1e-9,abstol=1e-9)
data = convert(Array, data_sol)

3×3002 Array{Float64,2}:
 1.0  0.917924    0.867919    0.84536     …  13.2896   12.5913    1.35608
 0.0  0.26634     0.51174     0.744654        6.71989   5.22868   1.68825
 0.0  0.00126393  0.00465567  0.00983655     39.1699   38.904    16.687  

Plot the data

In [7]:
# plot(data_sol_short,vars=(1,2,3)) # the short solution
# plot(data_sol,vars=(1,2,3)) # the longer solution
#interpolation_sol = solve(prob,Vern7(),saveat=t,reltol=1e-12,abstol=1e-12)
#plot(interpolation_sol,vars=(1,2,3))

In [8]:
#xyzt = plot(data_sol_short, plotdensity=10000,lw=1.5)
#xy = plot(data_sol_short, plotdensity=10000, vars=(:x,:y))
#xz = plot(data_sol_short, plotdensity=10000, vars=(:x,:z))
#yz = plot(data_sol_short, plotdensity=10000, vars=(:y,:z))
#xyz = plot(data_sol_short, plotdensity=10000, vars=(:x,:y,:z))
#plot(plot(xyzt,xyz),plot(xy, xz, yz, layout=(1,3),w=1), layout=(2,1), size=(800,600))

In [9]:
xyzt = plot(data_sol, plotdensity=10000,lw=1.5)
xy = plot(data_sol, plotdensity=10000, vars=(:x,:y))
xz = plot(data_sol, plotdensity=10000, vars=(:x,:z))
yz = plot(data_sol, plotdensity=10000, vars=(:y,:z))
xyz = plot(data_sol, plotdensity=10000, vars=(:x,:y,:z))
plot(plot(xyzt,xyz),plot(xy, xz, yz, layout=(1,3),w=1), layout=(2,1), size=(800,600))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 t 
 
<polyline clip-path="url(#clip7302)" style="stroke:#009af9; stroke-width:1.5; stroke-opacity:1; fill:none" points="
 35.8488,176.848 35.9177,177.134 35.9866,177.309 36.0555,177.388 36.1243,177.383 36.1932,177.303 36.2621,177.153 36.331,176.937 36.3999,176.657 36.4688,176.312 
 36.5377,175.9 36.6066,175.418 36.6755,174.861 36.7444,174.222 36.8133,173.494 36.8822,172.669 36.9511,171.736 37.02,170.684 37.0889,169.5 37.1577,168.17 
 37.2266,166.679 37.2955,165.009 37.3644,163.144 37.4333,161.066 37.5022,158.758 37.5711,156.203 37.64,153.389 37.7089,150.307 37.7778,146.957 37.8467,143.349 
 37.9156,139.509 37.9845,135.483 38.0534,131.343 38.1222,127.19 38.1911,123.158 38.26,119.412 38.3289,116.143 38.3978,113.553 38.4667,111.834 38.5356,111.15 
 38.6045,111.606 38.6734,113.235 38.7423,115.989 38.8112,119.747 38.8801,124.329 38.949,129.523 39.0179,135.108 39.0868,140.877 39.1556,146.65 39.2245,152.284 
 39.2934,157.669 39.3623,162.733 39.4312,167.43 39.5001,171.74 39.569,175.658 39.6379,179.196 39.7068,182.371 39.7757,185.207 39.8446,187.733 39.9135,189.975 
 39.9824,191.964 40.0513,193.726 40.1202,195.288 40.189,196.675 40.2579,197.909 40.3268,199.011 40.3957,199.999 40.4646,200.89 40.5335,201.698 40.6024,202.438 
 40.6713,203.12 40.7402,203.754 40.8091,204.348 40.878,204.911 40.9469,205.447 41.0158,205.962 41.0847,206.46 41.1536,206.944 41.2224,207.416 41.2913,207.877 
 41.3602,208.329 41.4291,208.771 41.498,209.204 41.5669,209.625 41.6358,210.035 41.7047,210.43 41.7736,210.809 41.8425,211.17 41.9114,211.51 41.9803,211.826 
 42.0492,212.116 42.1181,212.377 42.187,212.607 42.2558,212.802 42.3247,212.962 42.3936,213.083 42.4625,213.166 42.5314,213.208 42.6003,213.21 42.6692,213.17 
 42.7381,213.091 42.807,212.972 42.8759,212.816 42.9448,212.625 43.0137,212.401 43.0826,212.147 43.1515,211.867 43.2203,211.563 43.2892,211.241 43.3581,210.904 
 43.427,210.557 43.4959,210.203 43.5648,209.846 43.6337,209.491 43.7026,209.141 43.7715,208.801 43.8404,208.473 43.9093,208.16 43.9782,207.866 44.0471,207.594 
 44.116,207.344 44.1849,207.121 44.2537,206.924 44.3226,206.756 44.3915,206.618 44.4604,206.51 44.5293,206.434 44.5982,206.39 44.6671,206.377 44.736,206.396 
 44.8049,206.447 44.8738,206.529 44.9427,206.641 45.0116,206.783 45.0805,206.954 45.1494,207.151 45.2183,207.375 45.2871,207.622 45.356,207.892 45.4249,208.182 
 45.4938,208.49 45.5627,208.813 45.6316,209.149 45.7005,209.494 45.7694,209.846 45.8383,210.202 45.9072,210.557 45.9761,210.909 46.045,211.252 46.1139,211.585 
 46.1828,211.902 46.2517,212.2 46.3205,212.476 46.3894,212.725 46.4583,212.944 46.5272,213.13 46.5961,213.281 46.665,213.393 46.7339,213.465 46.8028,213.497 
 46.8717,213.486 46.9406,213.433 47.0095,213.338 47.0784,213.202 47.1473,213.027 47.2162,212.815 47.285,212.569 47.3539,212.292 47.4228,211.988 47.4917,211.66 
 47.5606,211.313 47.6295,210.95 47.6984,210.577 47.7673,210.198 47.8362,209.817 47.9051,209.438 47.974,209.066 48.0429,208.703 48.1118,208.355 48.1807,208.024 
 48.2496,207.713 48.3184,207.425 48.3873,207.162 48.4562,206.927 48.5251,206.72 48.594,206.544 48.6629,206.4 48.7318,206.288 48.8007,206.209 48.8696,206.164 
 48.9385,206.152 49.0074,206.174 49.0763,206.23 49.1452,206.317 49.2141,206.437 49.283,206.589 49.3518,206.77 49.4207,206.979 49.4896,207.216 49.5585,207.479 
 49.6274,207.765 49.6963,208.072 49.7652,208.398 49.8341,208.741 49.903,209.097 49.9719,209.464 50.0408,209.837 50.1097,210.215 50.1786,210.592 50.2475,210.966 
 50.3164,211.332 50.3852,211.685 50.4541,212.023 50.523,212.341 50.5919,212.634 50.6608,212.899 50.7297,213.133 50.7986,213.331 50.8675,213.491 50.9364,213.611 
 51.0053,213.688 51.0742,213.72 51.1431,213.708 51.212,213.65 51.2809,213.548 51.3498,213.402 51.4186,213.214 51.4875,212.

In [10]:
# Note: Euler uses tstops to hit the estimation timepoints exactly since it's not adaptive
# obj_short = build_loss_objective(prob_short,Euler(),L2Loss(t_short,data_short),tstops=t_short)
# res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 11e3)
# Euler could not recover the correct results since its error is too high!

In [11]:
obj_short = build_loss_objective(prob_short,Tsit5(),L2Loss(t_short,data_short),tstops=t_short)
res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 7e3)
# Tolernace is still too high to get close enough

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 552 evals, 402 steps, improv/step: 0.313 (last = 0.3134), fitness=6716.326361610
1.00 secs, 1202 evals, 1048 steps, improv/step: 0.293 (last = 0.2802), fitness=321.702484357
1.50 secs, 1896 evals, 1743 steps, improv/step: 0.283 (last = 0.2676), fitness=10.934223374
2.01 secs, 2582 evals, 2429 steps, improv/step: 0.284 (last = 0.2872), fitness=0.250119100
2.51 secs, 3271 evals, 3119 steps, improv/step: 0.289 (last = 0.3072), fitness=0.014049874
3.01 secs, 3960 evals, 3810 steps, improv/step: 0.291 (last = 0.2996), fitness=0.000606500
3.51 secs, 4649 evals, 4499 steps, improv/step: 0.295 (last = 0.3179), fitness=0.000015720
4.01 secs, 5336 evals, 5186 steps, improv/step: 0.295 (last = 0.2969), fitness=0.000000547


BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (7000) reached", 7001, 1.516009579208e9, 5.325000047683716, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 877408),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 7000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 7151, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,A

In [12]:
obj_short = build_loss_objective(prob_short,Tsit5(),L2Loss(t_short,data_short),tstops=t_short,reltol=1e-9)
res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 7e3)
# With the tolerance lower, it achieves the correct solution in 3.5 seconds.

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 547 evals, 437 steps, improv/step: 0.339 (last = 0.3387), fitness=756.296146700
1.00 secs, 1038 evals, 927 steps, improv/step: 0.313 (last = 0.2898), fitness=103.680126366
1.51 secs, 1496 evals, 1385 steps, improv/step: 0.287 (last = 0.2336), fitness=11.483581187
2.01 secs, 2015 evals, 1904 steps, improv/step: 0.288 (last = 0.2909), fitness=3.929016787
2.52 secs, 2570 evals, 2459 steps, improv/step: 0.289 (last = 0.2937), fitness=0.086793633
3.02 secs, 3116 evals, 3006 steps, improv/step: 0.293 (last = 0.3090), fitness=0.002534761
3.52 secs, 3672 evals, 3563 steps, improv/step: 0.293 (last = 0.2944), fitness=0.000480736
4.02 secs, 4226 evals, 4117 steps, improv/step: 0.289 (last = 0.2617), fitness=0.000085094
4.

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (7000) reached", 7001, 1.516009598509e9, 6.658999919891357, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 105042),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 7000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 7109, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,A

In [13]:
obj_short = build_loss_objective(prob_short,Vern9(),L2Loss(t_short,data_short),tstops=t_short,reltol=1e-9,abstol=1e-9)
res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 7e3)
# With the more accurate solver Vern9 in the solution of the ODE, the convergence is less efficient!

# Fastest BlackBoxOptim: 3.5 seconds

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 622 evals, 519 steps, improv/step: 0.362 (last = 0.3622), fitness=635.844639846
1.00 secs, 1228 evals, 1125 steps, improv/step: 0.316 (last = 0.2772), fitness=18.213355891
1.50 secs, 1843 evals, 1741 steps, improv/step: 0.310 (last = 0.2987), fitness=0.417039337
2.01 secs, 2459 evals, 2357 steps, improv/step: 0.297 (last = 0.2597), fitness=0.021587630
2.51 secs, 3072 evals, 2970 steps, improv/step: 0.288 (last = 0.2545), fitness=0.001723951
3.01 secs, 3690 evals, 3588 steps, improv/step: 0.288 (last = 0.2848), fitness=0.000323200
3.51 secs, 4305 evals, 4203 steps, improv/step: 0.286 (last = 0.2732), fitness=0.000013792
4.01 secs, 4924 evals, 4822 steps, improv/step: 0.288 (last = 0.3021), fitness=0.000000950
4.5

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (7000) reached", 7001, 1.516009612675e9, 5.903000116348267, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 69609),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 7000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 7103, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,Ar

# Using NLopt

First, the global optimization algorithms

In [14]:
obj_short = build_loss_objective(prob_short,Vern9(),L2Loss(t_short,data_short),tstops=t_short,reltol=1e-9,abstol=1e-9)

(::DiffEqObjective) (generic function with 2 methods)

In [15]:
opt = Opt(:GN_ORIG_DIRECT_L, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Accurate 3.2 seconds

  3.177196 seconds (18.55 M allocations: 828.775 MiB, 5.84% gc time)


(7.399386484909725e-18, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [16]:
opt = Opt(:GN_CRS2_LM, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Accurate 3.0 seconds

  2.728951 seconds (16.14 M allocations: 721.363 MiB, 5.90% gc time)


(2.852927598467924e-18, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [17]:
opt = Opt(:GN_ISRES, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Accurate to single precision 8.2 seconds

  8.101673 seconds (47.62 M allocations: 2.078 GiB, 5.55% gc time)


(0.0014260884142458603, [9.99928, 27.9999, 2.65975], :MAXEVAL_REACHED)

In [18]:
opt = Opt(:GN_ESCH, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Approximatively accurate, good starting values for local optimization

  8.145011 seconds (47.62 M allocations: 2.078 GiB, 5.71% gc time)


(238.30645522592377, [10.1912, 27.6588, 2.75838], :MAXEVAL_REACHED)

Next, the local optimization algorithms that could be used after the global algorithms as a check on the solution and its precision. All the local optimizers are started from LocIniPar and with the narrow bounds of the Xiang2015Paper. 

In [19]:
opt = Opt(:LN_BOBYQA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # 0.1 seconds

  0.104445 seconds (523.83 k allocations: 23.408 MiB, 4.82% gc time)


(2.76954149411742e-18, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [20]:
opt = Opt(:LN_NELDERMEAD, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.29 sec

  0.292511 seconds (1.56 M allocations: 69.599 MiB, 5.57% gc time)


(2.7730093540821814e-18, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [21]:
opt = Opt(:LD_SLSQP, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.21 sec

  0.199501 seconds (852.77 k allocations: 38.189 MiB, 6.29% gc time)


(1.1112469462930288e-15, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [22]:
opt = Opt(:LN_COBYLA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 1.84 sec

  1.807022 seconds (10.34 M allocations: 461.971 MiB, 5.95% gc time)


(2.861949697593622e-18, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [23]:
opt = Opt(:LN_NEWUOA_BOUND, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.18 sec ROUNDOFF LIMITED

  0.181245 seconds (742.88 k allocations: 33.196 MiB, 6.80% gc time)


(5.275299515678992e-10, [10.0, 28.0, 2.66], :SUCCESS)

In [24]:
opt = Opt(:LN_PRAXIS, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.18 sec

  0.174023 seconds (895.27 k allocations: 40.005 MiB, 6.60% gc time)


(2.809177227281428e-18, [10.0, 28.0, 2.66], :SUCCESS)

In [25]:
opt = Opt(:LN_SBPLX, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.65 sec

  0.627672 seconds (3.56 M allocations: 159.169 MiB, 5.70% gc time)


(2.7849937877089793e-18, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [26]:
opt = Opt(:LD_MMA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.7 sec

  0.683145 seconds (3.90 M allocations: 174.234 MiB, 6.04% gc time)


(2.4876525761108884e-16, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [27]:
opt = Opt(:LD_LBFGS, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.12 sec

  0.122369 seconds (599.95 k allocations: 26.806 MiB, 5.81% gc time)


(1.1160630883835927e-15, [10.0, 28.0, 2.66], :SUCCESS)

In [28]:
opt = Opt(:LD_TNEWTON_PRECOND_RESTART, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.15 sec

  0.155022 seconds (766.60 k allocations: 34.252 MiB, 8.19% gc time)


(1.1162058847408525e-15, [10.0, 28.0, 2.66], :SUCCESS)


## Now let's solve the longer version

Notice from the plotting above that this ODE problem is chaotic and tends to diverge over time. In the longer version of parameter estimation, the dataset is increased to 3000 observations per variable with the same integration time step of 0.01.

In [29]:
#obj = build_loss_objective(prob,Euler(),L2Loss(t,data),tstops=t)
#res1 = bboptimize(obj;SearchRange = LooserBounds, MaxSteps = 4e3)
# Once again, Euler fails to converge. Error cannot be reduced.

In [32]:
# BB with Vern9 converges very slowly. The final values are within the NarrowBounds.
obj = build_loss_objective(prob,Vern9(),L2Loss(t,data),tstops=t,reltol=1e-9,abstol=1e-9)

res1 = bboptimize(obj;SearchRange = LooserBounds, MaxSteps = 4e3) # Default adaptive_de_rand_1_bin_radiuslimited 33 sec [10.2183, 24.6711, 2.28969]
#res1 = bboptimize(obj;SearchRange = LooserBounds, Method = :adaptive_de_rand_1_bin, MaxSteps = 4e3) # Method 32 sec [13.2222, 25.8589, 2.56176]
#res1 = bboptimize(obj;SearchRange = LooserBounds, Method = :dxnes, MaxSteps = 2e3) # Method dxnes 119 sec  [16.8648, 24.393, 2.29119]
#res1 = bboptimize(obj;SearchRange = LooserBounds, Method = :xnes, MaxSteps = 2e3) # Method xnes 304 sec  [19.1647, 24.9479, 2.39467]
#res1 = bboptimize(obj;SearchRange = LooserBounds, Method = :de_rand_1_bin_radiuslimited, MaxSteps = 2e3) # Method 44 sec  [13.805, 24.6054, 2.37274]
#res1 = bboptimize(obj;SearchRange = LooserBounds, Method = :generating_set_search, MaxSteps = 2e3) # Method 195 sec [19.1847, 24.9492, 2.39412]

Starting optimization with optimizer BlackBoxOptim.DXNESOpt{Float64,BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
σ=1.0 η[x]=1.0 η[σ]=0.0 η[B]=0.0 |tr(ln_B)|=8.881784197001252e-16 |path|=0.0 speed=0.0
0.53 secs, 56 evals, 8 steps, fitness=607974.313059148
σ=0.3697861588719081 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.1102230246251565e-15 |path|=1.9253531076025812 speed=0.6878181419793957
1.04 secs, 119 evals, 17 steps, fitness=532754.572944665
σ=0.6450166499262784 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.2212453270876722e-15 |path|=0.9174074326878715 speed=0.32773701265384103
1.55 secs, 182 evals, 26 steps, fitness=527505.698751140
σ=0.15658991602931807 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.2212453270876722e-15 |path|=1.8905823597648288 speed=0.6753965497641492
2.06 secs, 245 evals, 35 steps, fitness=527505.698751140
σ=0.03320262694133043 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B

21.34 secs, 2415 evals, 345 steps, fitness=525712.471725659
σ=4.707292846508807e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.3322676295501878e-15 |path|=1.1435579449363538 speed=0.4085276086895618
21.87 secs, 2478 evals, 354 steps, fitness=525712.471725659
σ=4.221112916706386e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.3322676295501878e-15 |path|=2.2029819924387493 speed=0.7869990054655761
22.40 secs, 2541 evals, 363 steps, fitness=525712.471725659
σ=4.09039339156541e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.7763568394002505e-15 |path|=0.7077958860980995 speed=0.2528548396417807
22.96 secs, 2604 evals, 372 steps, fitness=525712.471725659
σ=1.7907218173779252e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=2.220446049250313e-15 |path|=0.7649825850181904 speed=0.2732843644088662
23.49 secs, 2667 evals, 381 steps, fitness=525712.471725659
σ=1.346090062473842e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=2.6645352591003

41.80 secs, 4872 evals, 696 steps, fitness=525712.471725659
σ=7.964555455976832e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=8.881784197001252e-16 |path|=2.278409409839994 speed=0.8139448918520109
42.32 secs, 4935 evals, 705 steps, fitness=525712.471725659
σ=5.561761247532694e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.3322676295501878e-15 |path|=0.41256331682564396 speed=0.147385190232048
42.84 secs, 4998 evals, 714 steps, fitness=525712.471725659
σ=6.957193411967061e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=8.881784197001252e-16 |path|=1.1958252316734492 speed=0.4271997098786149
43.36 secs, 5061 evals, 723 steps, fitness=525712.471725659
σ=6.125862945210193e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=8.881784197001252e-16 |path|=1.8046938572141116 speed=0.6447135181640979
43.88 secs, 5124 evals, 732 steps, fitness=525712.471725659
σ=1.2121836394740538e-8 η[x]=1.0 η[σ]=1.5384615384615383 η[B]=0.056 |tr(ln_B)|=1.332267629550187

62.12 secs, 7329 evals, 1047 steps, fitness=525712.471725658
σ=3.78342425949426e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=4.440892098500626e-15 |path|=2.1545487874592992 speed=0.7696966015961094
62.64 secs, 7392 evals, 1056 steps, fitness=525712.471725658
σ=7.324913726934456e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=3.552713678800501e-15 |path|=1.4396959543287662 speed=0.5143207198780836
63.20 secs, 7455 evals, 1065 steps, fitness=525712.471725658
σ=4.081562659744621e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=3.1086244689504383e-15 |path|=1.2454213943097747 speed=0.4449175717602219
63.77 secs, 7518 evals, 1074 steps, fitness=525712.471725658
σ=5.409620565693174e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=3.1086244689504383e-15 |path|=0.7409842107810002 speed=0.26471112290154397
64.28 secs, 7574 evals, 1082 steps, fitness=525712.471725658
σ=3.818689305928666e-9 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=3.552713678

82.90 secs, 9737 evals, 1391 steps, fitness=525712.471725658
σ=4.271629692423065e-11 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=7.993605777301127e-15 |path|=2.245260499937066 speed=0.8021026892305578
83.44 secs, 9800 evals, 1400 steps, fitness=525712.471725658
σ=7.97795790141745e-11 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=8.43769498715119e-15 |path|=1.2659422686527315 speed=0.4522485021784559
83.98 secs, 9863 evals, 1409 steps, fitness=525712.471725658
σ=3.510513951262372e-11 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=8.881784197001252e-15 |path|=0.7458332167425059 speed=0.2664433943782514
84.52 secs, 9926 evals, 1418 steps, fitness=525712.471725658
σ=4.4853138400423905e-11 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=8.881784197001252e-15 |path|=2.5258369439957056 speed=0.9023365464246296
85.05 secs, 9989 evals, 1427 steps, fitness=525712.471725658
σ=5.432645484883753e-11 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=1.02140518

103.59 secs, 12173 evals, 1739 steps, fitness=525712.471725658
σ=1.7547515811841337e-10 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=7.105427357601002e-15 |path|=0.8992773795524621 speed=0.32126018540986984
104.11 secs, 12236 evals, 1748 steps, fitness=525712.471725658
σ=1.2571483637601627e-10 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=6.661338147750939e-15 |path|=1.2422896480357375 speed=0.44379877859191763
104.62 secs, 12299 evals, 1757 steps, fitness=525712.471725658
σ=1.707870913097036e-10 η[x]=1.0 η[σ]=1.0 η[B]=0.104 |tr(ln_B)|=7.993605777301127e-15 |path|=2.935738306831664 speed=1.0487707733035188
105.14 secs, 12362 evals, 1766 steps, fitness=525712.471725658
σ=1.8474894556429236e-10 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=7.993605777301127e-15 |path|=0.8490428746937325 speed=0.3033142804957265
105.65 secs, 12425 evals, 1775 steps, fitness=525712.471725658
σ=1.0636752462153387e-10 η[x]=1.0 η[σ]=0.7692307692307692 η[B]=0.056 |tr(ln_B)|=8.437694

BlackBoxOptim.OptimizationResults("dxnes", "Max number of steps (2000) reached", 2001, 1.516010076532e9, 119.12100005149841, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 737015),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:Method, :dxnes),Pair{Symbol,Any}(:MaxSteps, 2000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 14007, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Floa

In [31]:
# using Evolutionary
# N = 3
# @time result, fitness, cnt = cmaes(obj, N; μ = 3, λ = 12, iterations = 1000) # cmaes( rastrigin, N; μ = 15, λ = P, tol = 1e-8)

In [33]:
opt = Opt(:GN_ORIG_DIRECT_L, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Fail to converge 

 14.938654 seconds (101.87 M allocations: 3.651 GiB, 5.28% gc time)


(470363.2181996153, [7.04676, 23.6656, 1.80658], :XTOL_REACHED)

In [36]:
opt = Opt(:GN_CRS2_LM, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 20000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Hit and miss. converge approximately accurate values for local opt.91 seconds

153.596003 seconds (1.06 G allocations: 37.816 GiB, 5.30% gc time)


(525967.3577305243, [19.1637, 24.9451, 2.39478], :MAXEVAL_REACHED)

In [35]:
opt = Opt(:GN_ISRES, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 50000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Approximately accurate within local bounds

389.922834 seconds (2.64 G allocations: 94.539 GiB, 5.63% gc time)


(457440.01339330774, [9.2112, 27.0372, 2.223], :MAXEVAL_REACHED)

In [37]:
opt = Opt(:GN_ESCH, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 20000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Approximately accurate

166.449863 seconds (1.06 G allocations: 37.816 GiB, 5.59% gc time)


(452355.1761286359, [9.23828, 26.8303, 2.27546], :MAXEVAL_REACHED)

This parameter estimation on the longer sample proves to be extremely challenging for the global optimizers. BlackBoxOptim is best in optimizing the objective function. All of the global algorithms produces final parameter estimates that could be used as starting values for further refinement with the local optimization algorithms.

In [56]:
opt = Opt(:LN_BOBYQA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Claims SUCCESS but does not iterate to the true values.

  2.266360 seconds (5.70 M allocations: 209.106 MiB, 2.40% gc time)


(588301.876527702, [9.86631, 20.5815, 2.0], :SUCCESS)

In [57]:
opt = Opt(:LN_NELDERMEAD, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-9)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Inaccurate final values

  3.962906 seconds (9.65 M allocations: 354.319 MiB, 2.02% gc time)


(439664.3609672592, [9.9937, 23.2594, 2.15897], :XTOL_REACHED)

In [58]:
opt = Opt(:LD_SLSQP, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Inaccurate final values

  1.227923 seconds (3.01 M allocations: 110.359 MiB, 2.07% gc time)


(591291.6479752676, [9.1791, 20.6017, 2.0338], :XTOL_REACHED)

No local optimizer can improve the global solution to the true values.

# Conclusion:

 1) To benchmark parameter estimators, make sure the synthetic data is correct.
    The lorenz_test.jl results are all skewed because the actual Euler solution
    producing the synthetic data was skewed. Euler being a fixed timestep, it was able
    to rediscover the parameters that built the data, given that it's hitting all the
    same points. In this experiemnt this is no longer the case. There is adaptive timestepping with Vern7,
    we're getting rid of the ability to "overfit", i.e. simply match the data
    we generated. Instead, we have to hit the same solution, albeit at different
    stepping points, making it less prone to generating bias.
    
 2) As expected the Lorenz system is extremely sensitive to initial space values. 
    Starting the integration from r0 = [0.1,0.0,0.0] produces convergence with the short
    sample of 300 observations. This can be achieved by all the global optimizers as well as most of the local optimizers. 
    Instead starting from r0= [-11.8,-5.1,37.5], as in PODES, with the shorter sample shrinks
    the number of successful algorithms to 3: BBO, :GN_CRS2_LM and :LD_SLSQP. For the longer sample, all the algorithms fail.
    THIS IS A FAILURE OF THE NONLINEAR LEAST SQUARES ESTIMATOR L2LOSS.
    
 3) When trying to hit the real data, having a low enough tolerance on the
    numerical solution is key. If the numerical solution is too rough, then
    we can never actually hone in on the true parameters since even with the
    true parameters we will erroneously induce numerical error. Maybe this
    could be adaptive?
    
 4) Excessively low tolerance in the numerical solution is inefficient and delays the convergence of the estimation.
 
 5) The estimation method and the glocal versus local optimization make a huge difference in the timings. Here, BBO
    always find the correct solution for a global optimization setup. For local optimization, most methods in NLopt,
    like :LN_BOBYQA, solve the problem in <0.05 seconds. This is an algorithm that can scale a local optimization
    but we are aiming to scale a global optimization.
    
 6) Fitting shorter timespans is easier... maybe this can lead to determining a minimal sample size for the 
    optimizers and the estimator to succeed.